In [1]:
import numpy as np
import pandas as pd
data = pd.read_csv("FDX_4_year_training_signal.csv")
dt=data
data.drop(data.columns[[0]],axis=1,inplace=True)
window_size = int(len(data)/(12))
splittrain = []
splittest = [] 
for i in range(0, len(data)-3*window_size,window_size):
    splittrain.append(data[i:i+3*window_size])
    splittest.append(data[i+3*window_size:i+4*window_size])

In [2]:
from scipy.optimize import NonlinearConstraint, Bounds, LinearConstraint
import sympy as sp
w1=sp.symbols('w1')
w2=sp.symbols('w2')
w3=sp.symbols('w3')
w4=sp.symbols('w4')
w5=sp.symbols('w5')
w6=sp.symbols('w6')
w7=sp.symbols('w7')
w8=sp.symbols('w8')
w9=sp.symbols('w9')
w10=sp.symbols('w10')
td=sp.symbols('td')

In [3]:
def const(p):
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    return w1+w2+w3+w4+w5+w6+w7+w8+w9+w10
nlc = NonlinearConstraint(const, 1 ,1)
import numpy as np
from scipy.optimize import differential_evolution
from scipy.optimize import NonlinearConstraint, Bounds

bounds = [[0, 1], [0, 1],[0, 1], [0, 1],[0, 1],[0, 1],[0, 1], [0, 1],[0, 1],[0,1],[0,0.5]]

In [4]:
coll=[]

def func(p):
    global S,B,resl,resfl
    S=0
    B=0
    resl=[]
    resfl=[]
    w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,td = p
    mon=100000
    amount=0
    for i in range(0,len(dfo)):
        r = ((w1*sl1[i]+w2*sl2[i]+w3*sl3[i]+w4*sl4[i]+w5*sl5[i]+w6*sl6[i]+w7*sl7[i]+w8*sl8[i]+w9*sl9[i]+w10*sl10[i])/(w1+w2+w3+w4+w5+w6+w7+w8+w9+w10))#w1=sp.symbols('w1')
        if i!=0:
            if dfo['adj_close_price'][i]<dfo['adj_close_price'][i-1]*0.8:
                mon+=amount*dfo['adj_close_price'][i]*0.998
                amount=0
        if r>td:
            sumall=mon//dfo['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*dfo['adj_close_price'][i]*1.002
            B+=1
        elif r<-td:
            mon+=amount*dfo['adj_close_price'][i]*0.998
            amount=0
            S+=1
        else:
            pass
    ret=mon+amount*dfo['adj_close_price'][len(dfo)-1]
    return -((ret-100000)/1000)
def fobj():
    global sl1,sl2,sl3,sl4,sl5,sl6,sl7,sl8,sl9,sl10,dfo
    for i in range(len(splittrain)):
        dfo=splittrain[i]
        dfo=dfo.reset_index()
        sl1=dfo['renko_macd_signal']
        sl2=dfo['bb_rsi_signal']
        sl3=dfo['aroon_signal']
        sl4=dfo['obv_macd_signal']
        sl5=dfo['ma50_200_signal']
        sl6=dfo['ma30_200_signal']
        sl7=dfo['adx_signal']
        sl8=dfo['stoch_adx_signal']
        sl9=dfo['cdc_signal']
        sl10=dfo['psar_wpr_signal']
        result = differential_evolution(func, bounds,constraints=(nlc),maxiter=3000,popsize=80,recombination=0.8) 
        coll.append([result.x,result.fun,i])
        print([result.x,result.fun,i])

In [5]:
fobj()

C:\Users\Windows 11\anaconda3\lib\site-packages\scipy\optimize\_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


[array([0.35581553, 0.00459294, 0.02755227, 0.1290859 , 0.04575189,
       0.10942361, 0.09756681, 0.08749956, 0.00820624, 0.13450526,
       0.49489581]), -0.0, 0]
[array([0.02748048, 0.06284215, 0.17547358, 0.13829312, 0.24993537,
       0.02108198, 0.06295895, 0.2138095 , 0.03657947, 0.0115454 ,
       0.16753558]), -12.059959877593966, 1]
[array([0.23813597, 0.1673876 , 0.01095478, 0.04725356, 0.05464635,
       0.19752492, 0.00607644, 0.21807154, 0.04018342, 0.01976542,
       0.44561761]), -0.0, 2]
[array([0.01297013, 0.03551081, 0.02060555, 0.00808516, 0.06848273,
       0.04937194, 0.1327122 , 0.4387661 , 0.06525923, 0.16823615,
       0.24520921]), -0.0, 3]
[array([0.16407677, 0.01231474, 0.12162325, 0.05410559, 0.009329  ,
       0.54207658, 0.07993898, 0.00608363, 0.00325584, 0.00719562,
       0.35628527]), -76.75227286706544, 4]
[array([1.40156844e-02, 7.89539528e-02, 3.03719687e-01, 3.59698741e-01,
       7.20583283e-03, 2.36509972e-04, 2.47081774e-02, 2.08971476e-03,
   

In [8]:
#test
sums=0
listna=[]
for j in range(0,11): #เเต่ละ signal 
    for i in range(0,10): #เเต่ละ data
        sums+=coll[i][0][j]
    sums=sums/10
    listna.append(sums)
tdna=listna[10]
w=listna[0:10]
dfo=pd.read_csv('FDX_1_year_testing_signal.csv')
count1,count2,count3=0,0,0
decision_d_l=[]
il=[]
sell_idx,buy_idx=[],[]
def testing(td,w,df):
    global count1,count2,count3,sell_idx,buy_idx,sl1
    mon=100000
    amount=0
    sumall=0
    sl1=dfo['renko_macd_signal']
    sl2=dfo['bb_rsi_signal']
    sl3=dfo['aroon_signal']
    sl4=dfo['obv_macd_signal']
    sl5=dfo['ma50_200_signal']
    sl6=dfo['ma30_200_signal']
    sl7=dfo['adx_signal']
    sl8=dfo['stoch_adx_signal']
    sl9=dfo['cdc_signal']
    sl10=dfo['psar_wpr_signal']
    
    for i in range(0,len(df)):
        decision_d= ((w[0]*sl1[i]+w[1]*sl2[i]+w[2]*sl3[i]+w[3]*sl4[i]+w[4]*sl5[i]+w[5]*sl6[i]+w[6]*sl7[i]+w[7]*sl8[i]+w[8]*sl9[i]+w[9]*sl10[i])/sum(w))
        if i!=0:
            if df['adj_close_price'][i]<df['adj_close_price'][i-1]*0.2:
                    mon+=amount*dfo['Close'][i]*0.998
                    amount=0
                    sell_idx.append(i)
        if decision_d>td:
            sumall=mon//df['adj_close_price'][i]
            amount+=sumall
            mon-=sumall*df['adj_close_price'][i]*1.002
            count1+=1
            buy_idx.append(i)
        elif decision_d<-td:
            mon+=amount*df['adj_close_price'][i]*0.998
            amount=0
            count2+=1
            sell_idx.append(i)
        else:
            count3+=1
        decision_d_l.append(mon+amount*df['adj_close_price'][i])
        il.append(i)
    ret=mon+amount*df['adj_close_price'][len(df)-1]
    return (ret-100000)/1000
testing(tdna,w,dfo)